In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
# pickle_file = 'SVHN.pickle'
pickle_file = 'toy_data.pickle'
with open(pickle_file, 'rb') as inFile:
    content = pickle.load(inFile)
    X_train = content['toy_X_train'] # content['X_train']
    y_train = content['toy_y_train'] # content['y_train']
    X_valid = content['toy_X_valid'] # content['X_valid']
    y_valid = content['toy_y_valid'] # content['y_valid']
    X_test = content['toy_X_test'] # content['X_test']
    y_test = content['toy_y_test'] # content['y_test']
    del content
    print('Training set', X_train.shape, y_train.shape)
    print('Validation set', X_valid.shape, y_valid.shape)
    print('Test set', X_test.shape, y_test.shape)

Training set (10000, 32, 32, 1) (10000, 6)
Validation set (800, 32, 32, 1) (800, 6)
Test set (1000, 32, 32, 1) (1000, 6)


In [3]:
print(y_test.dtype, y_train.dtype, y_valid.dtype)

float32 float32 float32


In [4]:
y_test = y_test.astype('int32', copy=False)
y_train = y_train.astype('int32', copy=False)
y_valid = y_valid.astype('int32', copy=False)

In [5]:
print(y_test.dtype, y_train.dtype, y_valid.dtype)

int32 int32 int32


In [6]:
def accuracy(predictions, labels):
    
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [20]:
img_size = 32
n_labels = 10
n_channels = 1
batch_size = 64

kernel_size = 3
depth1, depth2 = 16, 32
n_nodes_full = 128
graph = tf.Graph()
with graph.as_default():
    
    # train, validation and test data
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, img_size, img_size, n_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape = (batch_size, 6))
    tf_valid_dataset = tf.constant(X_valid)
    tf_test_dataset = tf.constant(X_test)
    
    # variables
    w1_conv = tf.get_variable('conv1',
                                   shape=[kernel_size,kernel_size,n_channels,depth1],
                                   initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b1_conv = tf.Variable(tf.constant(1.0, shape=[depth1], name='conv1_biases'))
    
    
    w2_conv = tf.get_variable('conv2',
                                   shape=[kernel_size,kernel_size,depth1,depth2],
                                   initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b2_conv = tf.Variable(tf.constant(1.0, shape=[depth2], name='conv2_biases'))
   
    
    w_fc = tf.get_variable('fc_weights', shape=[8*8*32, n_nodes_full],
                             initializer=tf.contrib.layers.xavier_initializer())
    b_fc = tf.Variable(tf.constant(1.0, shape=[n_nodes_full], name='fc_biases'))
    
    
    w1_out = tf.get_variable('out_layer1', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    b1_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer1_biases'))
    
    
    w2_out = tf.get_variable('out_layer2', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    b2_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer2_biases'))
    
    
    
    w3_out = tf.get_variable('out_layer3', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    b3_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer3_biases'))
    
    
    w4_out = tf.get_variable('out_layer4', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    b4_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer4_biases'))
    
    
    w5_out = tf.get_variable('out_layer5', shape=[n_nodes_full, n_labels],
                             initializer=tf.contrib.layers.xavier_initializer())
    b5_out = tf.Variable(tf.constant(1.0, shape=[n_labels], name='out_layer5_biases'))
    
    
    
    def model(dataset, keep_prob):
        
        conv_1 = tf.nn.conv2d(dataset, w1_conv, [1,1,1,1],'SAME') + b1_conv
        relu_1 = tf.nn.relu(conv_1)
        pool_1 = tf.nn.max_pool(relu_1, [1,2,2,1], [1,2,2,1], 'SAME')
        conv_2 = tf.nn.conv2d(pool_1, w2_conv, [1,1,1,1], 'SAME') + b2_conv
        relu_2 = tf.nn.relu(conv_2)
        #print('drop-out layer shape is', drop.get_shape().as_list())
        pool_2 = tf.nn.max_pool(relu_2, [1,2,2,1], [1,2,2,1], 'SAME')
        pool_2 = tf.reshape(pool_2, (-1, 8*8*32))
        # # print('after reshape, the shape is: ', pool_2.get_shape().as_list())
        # print('full connected layer shape is:', w_fc.get_shape().as_list())
        fc = tf.nn.relu(tf.matmul(pool_2, w_fc) + b_fc)
        fc_dropped = tf.nn.dropout(fc, keep_prob)
        out_layers = (
            (w1_out, b1_out), (w2_out, b2_out), (w3_out, b3_out), (w4_out, b4_out), (w5_out, b5_out))
        # logits_1 = tf.matmul(fc_droped, w1_out) + b1_out
        # logits_2 = tf.matmul(fc_droped, w2_out) + b2_out
        # logits_3 = tf.matmul(fc_droped, w3_out) + b3_out
        # logits_4 = tf.matmul(fc_droped, w4_out) + b4_out
        # logits_5 = tf.matmul(fc_droped, w5_out) + b5_out
        # print('logits1 shape is', logits_5.get_shape().as_list())
        logits = []
        for i, layer in enumerate(out_layers):
            logit = tf.matmul(fc_dropped, layer[0]) + layer[1]
            logits.append(logit)
        
        return logits
    
    # training computation.
    logits = model(tf_train_dataset, 1.0)
    # print('logit shape is', logits[4].get_shape().as_list())
    # print('train label shape is',tf_train_labels[:,5].get_shape().as_list())
    loss_1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[0], tf_train_labels[:,1]))
    loss_2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[1], tf_train_labels[:,2]))
    loss_3 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[2], tf_train_labels[:,3]))
    loss_4 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[3], tf_train_labels[:,4]))
    loss_5 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[4], tf_train_labels[:,5]))

    loss = loss_1 + loss_2 + loss_3 + loss_4 + loss_5
    
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = 0.01 # tf.train.exponential_decay(0.01, global_step, 10000, 0.93)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    
    # Predictions of train, validation and test datasets
    train_logits = model(tf_train_dataset, 0.5)
    train_prediction = tf.pack(map(tf.nn.softmax, train_logits))

    valid_logits = model(tf_valid_dataset, 1.0)
    valid_prediction = tf.pack(map(tf.nn.softmax, valid_logits))

    test_logits = model(tf_test_dataset, 1.0)
    test_prediction = tf.pack(map(tf.nn.softmax, test_logits))
    
    # valid_preds = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    # test_preds = tf.nn.softmax(model(tf_test_dataset, 1.0))
    
    saver = tf.train.Saver()

In [22]:
num_steps = 801 # 100001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  
    # reader = tf.train.NewCheckpointReader("CNN_1.ckpt")
    # reader.get_variable_to_shape_map()
    # saver.restore(session, "CNN_1.ckpt")
    # print("Model restored.")

    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :, :, :]
        batch_labels = y_train[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 80 == 0): 
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.5f%%' % accuracy(predictions, batch_labels[:,1:]))
            print('Validation accuracy: %.5f%%' % accuracy(valid_prediction.eval(), y_valid[:,1:]))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test[:,1:]))
    # save_path = saver.save(session, "CNN_multi.ckpt")
    # print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 72.930641
Minibatch accuracy: 3.43750%
Validation accuracy: 3.82500%
Minibatch loss at step 80: 5.917111
Minibatch accuracy: 7.81250%
Validation accuracy: 8.82500%
Minibatch loss at step 160: 4.845601
Minibatch accuracy: 7.18750%
Validation accuracy: 8.80000%
Minibatch loss at step 240: 5.180029
Minibatch accuracy: 7.18750%
Validation accuracy: 8.95000%
Minibatch loss at step 320: 4.878001
Minibatch accuracy: 4.06250%
Validation accuracy: 8.55000%
Minibatch loss at step 400: 5.110351
Minibatch accuracy: 8.12500%
Validation accuracy: 8.40000%
Minibatch loss at step 480: 5.306325
Minibatch accuracy: 5.62500%
Validation accuracy: 8.32500%
Minibatch loss at step 560: 6.842296
Minibatch accuracy: 8.43750%
Validation accuracy: 7.25000%
Minibatch loss at step 640: 20.386744
Minibatch accuracy: 6.56250%
Validation accuracy: 9.15000%
Minibatch loss at step 720: 97.790588
Minibatch accuracy: 5.62500%
Validation accuracy: 8.80000%
Minibatch loss at step 800: 